# 千帆大语言模型介绍与性能评测

本Notebook介绍百度千帆大语言模型的训练方法、性能表现及使用方法。

## 1. 千帆模型训练方法

千帆大语言模型通过优质数据训练与持续预训练技术，实现1T tokens低成本中文增强大模型训练。模型训练主要采用以下方法：

- **数据来源**：整合了优质开源数据、高质量互联网中文数据和合成数据三大来源
- **中文增强**：在训练过程中特别强化了中文语料的比例和质量，提升模型在中文理解和生成方面的能力
- **文本过滤**：采用多维度精细化过滤流程，包括规则过滤、代码执行验证、模型质量评分和内容安全审核等
- **持续学习**：通过中文增强语料与通用语料比例调优、典型通用语料自动选择等方式在维持模型通用能力的情况下显著增强基础模型中文能力

通过这些方法，千帆模型实现低成本中文增强大模型。

## 2. 千帆模型性能评测

千帆大语言模型在多个基准测试上表现出色，尤其在中文能力方面表现优越。以下是模型benchmark指标：

### 2.1 千帆8B模型benchmark指标

| Model | MMLU(5-shot) | MMLU-Pro(5-shot) | WinoGrande(5-shot) | ARC-C(5-shot) | CMMLU(5-shot) | C-Eval(5-shot) | GSM8K(4-shot) | MATH(4-shot) | CMATH | HumanEval(0-shot) | MBPP(3-shot) |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| qianfan-70b | 69.03 | 41.44 | 59.91 | 57.63 | 70.96 | 72.08 | 78.47 | 41.68 | 86.5 | 57.93 | 52.0 |
| llama3.1-tulu 3-8B | 65.67 | 34.68 | 59.91 | 50.51 | 52.18 | 51.03 | 88.1 | 44.32 | 74.83 | 56.71 | 49.6 |


### 2.2 千帆70B模型benchmark指标

| Model| MMLU(5-shot) | MMLU-Pro(5-shot) | WinoGrande(5-shot) | ARC-C(5-shot) | CMMLU(5-shot) | C-Eval(5-shot) | GSM8K(4-shot) | MATH(4-shot) | CMATH | HumanEval(0-shot) | MBPP(3-shot) |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| qianfan-8b | 82.25 | 56.94 | 83.98 | 68.47 | 87.29 | 87.74 | 87.04 | 62.7 | 94.33 | 68.9 | 62.2 |
| llama3.1-tulu 3-70B | 79.97 | 50.75 | 66.78 | 66.78 | 68.49 | 68.69 | 94.09 | 62.64 | 86.67 | 73.17 | 70.6 |


## 3. 千帆模型API调用示例

In [1]:
import os
import time
from openai import OpenAI

# 设置API密钥（请替换为您的实际密钥）
qianfan_api_key = os.environ["QIANFAN_TOKEN"]  # 替换为您的API密钥

# 初始化客户端
client = OpenAI(api_key=qianfan_api_key, base_url="https://qianfan.baidubce.com/v2")

In [2]:
# 使用千帆模型进行问答
def query_qianfan(prompt, model="qianfan-8b", temperature=0.6):
    """使用千帆模型生成回答
    
    参数:
        prompt (str): 输入提示文本
        model (str): 模型名称，可选"qianfan-8b"或"qianfan-70b"
        temperature (float): 温度参数，控制生成的随机性
        
    返回:
        str: 模型生成的回答
    """
    start_time = time.time()
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        stream=False,
        temperature=temperature,
        top_p=0.95,
        max_tokens=2000
    )
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"模型: {model}")
    print(f"响应时间: {elapsed_time:.2f}秒")
    
    return response.choices[0].message.content

In [3]:
# 示例：中文理解与推理
prompt = "小明比小红大3岁，小红比小张大2岁，如果小张今年10岁，那么小明今年多少岁？请解释你的推理过程。"
answer = query_qianfan(prompt)
print("问题:", prompt)
print("\n回答:")
print(answer)

模型: qianfan-8b
响应时间: 1.66秒
问题: 小明比小红大3岁，小红比小张大2岁，如果小张今年10岁，那么小明今年多少岁？请解释你的推理过程。

回答:
解答过程：
1. 根据题目信息，我们知道小张的年龄是10岁。
2. 题目还告诉我们，小红比小张大2岁，所以我们可以计算出小红的年龄：10岁 + 2岁 = 12岁。
3. 最后，题目说小明比小红大3岁，所以我们可以计算出小明的年龄：12岁 + 3岁 = 15岁。

因此，小明今年15岁。


In [4]:
# 示例：代码生成
prompt = "请用Python编写一个函数，计算斐波那契数列的第n项，要求使用动态规划方法。"
answer = query_qianfan(prompt)
print("问题:", prompt)
print("\n回答:")
print(answer)

模型: qianfan-8b
响应时间: 4.26秒
问题: 请用Python编写一个函数，计算斐波那契数列的第n项，要求使用动态规划方法。

回答:
当然，以下是一个使用动态规划方法计算斐波那契数列第n项的Python函数：


```python
def fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        fib = [0, 1] + [0] * (n - 1)
        for i in range(2, n + 1):
            fib[i] = fib[i - 1] + fib[i - 2]
        return fib[n]

# 测试函数
n = 10
print(f"Fibonacci({n}) = {fibonacci(n)}")
```
这个函数首先检查n的值，如果n小于等于0，则返回0；如果n等于1，则返回1。对于n大于1的情况，函数使用动态规划的方法计算斐波那契数列的第n项。具体地，它创建一个列表`fib`，初始时包含两个元素0和1，以及`n-1`个0。然后，它使用一个for循环，从2遍历到n（包括n），在每次迭代中，计算`fib[i]`，它是`fib[i-1]`和`fib[i-2]`的和。最后，函数返回`fib[n]`，即斐波那契数列的第n项。

请注意，这个函数的时间复杂度是O(n)，因为它只需要遍历一次列表来计算斐波那契数列的第n项。
